In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator

In [39]:
def loaddata():
    dataSet = [[0, 0,0,0, 'no'],
               [0, 0,0,1,'no'],
               [0, 1,0,1, 'yes'],
               [0, 1,1,0, 'yes'],
               [0, 0,0,0, 'no'],
               [1, 0,0,0, 'no'],
               [1, 0,0,1, 'no'],
               [1, 1,1,1, 'yes'],
               [1, 0,1,2, 'yes'],
               [1, 0,1,2, 'yes'],
               [2, 0,1,2, 'yes'],
               [2, 0,1,1, 'yes'],
               [2, 1,0,1, 'yes'],
               [2, 1,0,2, 'yes'],
               [2, 0,0,0,'no']]
    feature_name = ['age','job','house','credit']
    return dataSet, feature_name

### 计算数据集的熵

<img src="images/02.png" width="200" align="left"/>

In [40]:
def entropy(dataSet):
    #数据集条数
    n = len(dataSet)
    LabelCounts =  {}
    for i in range(n):
        label = dataSet[i][-1]
        if label not in LabelCounts.keys():
            LabelCounts[label] = 0
        LabelCounts[label] += 1

    #计算熵
    entropy = 0.0
    for key in LabelCounts:
        prob = float(LabelCounts[key])/n
        entropy -= prob * np.log2(prob)
    return  entropy

# 划分数据集

In [41]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

# 测试

In [42]:
dataSet, feature_name = loaddata()
splitDataSet(dataSet, 0, 0)


[[0, 0, 0, 'no'],
 [0, 0, 1, 'no'],
 [1, 0, 1, 'yes'],
 [1, 1, 0, 'yes'],
 [0, 0, 0, 'no']]

### 选择最好的特征
<img src="images/03.png" width="400" align="left"/>

In [43]:
# 计算信息增益
def chooseBestFeature(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = entropy(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]

        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * entropy(subDataSet)
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature


# 测试

In [44]:
chooseBestFeature(dataSet)

2

# 类别的投标表决

In [45]:
def classVotes(dataSet):
    #定义一个类别计数器，键为类别，值为类别的个数，寻找出出现次数最多的类别
    classCount = {}
    for vec in dataSet:
        label = vec[-1]
        if label not in classCount.keys():
            classCount[label] = 0
        classCount[label] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)#降序（itermgetter用法）
    return sortedClassCount[0][0]

# 递归训练一棵树

In [46]:
def createTree(dataSet, feature_name):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return classVotes(dataSet)
    bestFeat = chooseBestFeature(dataSet)
    bestFeatName = feature_name[bestFeat]
    myTree = {bestFeatName:{}}
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = feature_name[:]
        myTree[bestFeatName][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

In [47]:
myDat, feature_name = loaddata()
myTree = createTree(myDat, feature_name)
print(myTree)

{'house': {0: {'job': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


# 预测

In [50]:
def predict(inputTree,featLabels,testVec):#输入树，特征标签，测试向量
    print('inputTree.keys()=', inputTree.keys())

    firstStr = list(inputTree.keys())[0]
    print('firstStr=', firstStr)
    secondDict = inputTree[firstStr]
    print('secondDict=', secondDict)
    featIndex = featLabels.index(firstStr)
    print('featIndex=', featIndex)
    key = testVec[featIndex]
    print('key=', key)
    valueOfFeat = secondDict[key]
    print('-'*20)
    if isinstance(valueOfFeat, dict):
        classLabel = predict(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

In [51]:
print(predict(myTree, feature_name, [1,1,0,1]))

inputTree.keys()= dict_keys(['house'])
firstStr= house
secondDict= {0: {'job': {0: 'no', 1: 'yes'}}, 1: 'yes'}
featIndex= 2
key= 0
--------------------
inputTree.keys()= dict_keys(['job'])
firstStr= job
secondDict= {0: 'no', 1: 'yes'}
featIndex= 1
key= 1
--------------------
yes
